## SETI end-to-end narrowband drifting tone detection over multiple raw files
This notebook sequences through multiple raw files and looks for drifting narrowband tones.  

This is a general-purpose notebook for real observations such as Voyager or commensal pointings.

For each test file, notebook "01_seti_end_to_end.ipynb" is called which performs the following:
1) Inputs each RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor n_sti
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)

Note that these sets of files can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

The raw files are copied locally from bulk storage if needed, or rawspec can be run directly from bulk storage without copying locally. 
If local space is limited, raw files can be deleted after each run of this notebook. 

Plots are generated in each "01_seti_end_to_end.ipynb" call.

In [ ]:
%load_ext autoreload
%autoreload 2

parameters_are_undefined = False     # use parameter defaults settings in 01_seti_end_to_end.ipynb

delete_raw_file_after_use = True

display_figs01 = False
plot_dets1 = True
plot_dets2 = True
verbose = True
plot_sg_psd_zoom = True
plot_psd_zoom = False
plot_sg_zoom = False
plot_sg_wb = True
plot_psd_wb = True
zoom_bw_MHz = .012  # +/- 6 KHz
max_n_zoom_plots = 50

run_turbo = False
fb_ext = '.h5'

search_max_drift = 10    # Hz/sec
search_min_drift = -10   # ignored except for plotting

delete_raw_file_enable = True   # =True will delete raw file from $RAWDIR, if not present initially in $RAWDIR,
                                # and copied from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False will keep raw file in $RAWDIR regardless
delete_h5_sg_file = True        # =True h5 sg file will be deleted after finishing (reduce file space in $SGDIR)

enable_raw_backup_copy = False  # =True copy to $RAWDIR if needed from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False rawspec will read directly from $RAW_BACKUP_BASE_DIR/raw_backup_subdir without file copy

# settings for GBT, override otherwise
fine_fft_size = 1024*1024
n_sti = 4

if (1):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015'] # 23.7 dB, -.504 Hz/sec
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (1):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc3_guppi_57386_VOYAGER1_0004'] 
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (1):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_80036_DIAG_VOYAGER-1_0011', # 24.4 dB, -.279 Hz/sec
                        'blc23_guppi_59046_80354_DIAG_VOYAGER-1_0012', # no det for 0000
                        'blc23_guppi_59046_80672_DIAG_VOYAGER-1_0013', # 20.93 dB, -.52 Hz/sec
                        'blc23_guppi_59046_80989_DIAG_VOYAGER-1_0014', # no det for 0000
                        'blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015', # 26 dB, -.44 Hz/sec Up to 14 files!
                        'blc23_guppi_59046_81628_DIAG_VOYAGER-1_0016'] # no det for 0000
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (1):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0010',
                          'blc2_guppi_57388_HIP113357_0010',
                          'blc3_guppi_57388_HIP113357_0010',
                          'blc4_guppi_57388_HIP113357_0010',
                          'blc5_guppi_57388_HIP113357_0010',
                          'blc6_guppi_57388_HIP113357_0010'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (1):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0012',
                          'blc2_guppi_57388_HIP113357_0012',
                          'blc3_guppi_57388_HIP113357_0012',
                          'blc4_guppi_57388_HIP113357_0012',
                          'blc5_guppi_57388_HIP113357_0012',
                          'blc6_guppi_57388_HIP113357_0012'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_HIP/'
    raw_file_stem_list = ['blc40_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc41_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc42_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc43_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc44_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc45_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc46_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc47_guppi_59103_03394_DIAG_HIP95631_0015'] 
    fig_dir = './raw_gbt_59103_HIP/'
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_Kepler/'
    raw_file_stem_list = ['blc40_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc41_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc42_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc44_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc46_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc47_guppi_59103_01984_DIAG_KEPLER-160_0010'] 
    fig_dir = './raw_gbt_59103_Kepler/'
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_TIC/'
    raw_file_stem_list = ['blc40_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc41_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc42_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc43_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc44_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc45_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc46_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc47_guppi_59103_03721_DIAG_TIC159107668_0016'] 
    fig_dir = './raw_gbt_59103_TIC/'

n_raw_file = len(raw_file_stem_list)

print('raw_file_stem_list=')
for i_raw in range(n_raw_file):
    print(f'{raw_file_stem_list[i_raw]}')
             
print(f'Figures will appear in in {fig_dir}')

raw_file_stem_list=
blc40_guppi_59103_03721_DIAG_TIC159107668_0016
blc41_guppi_59103_03721_DIAG_TIC159107668_0016
blc42_guppi_59103_03721_DIAG_TIC159107668_0016
blc43_guppi_59103_03721_DIAG_TIC159107668_0016
blc44_guppi_59103_03721_DIAG_TIC159107668_0016
blc45_guppi_59103_03721_DIAG_TIC159107668_0016
blc46_guppi_59103_03721_DIAG_TIC159107668_0016
blc47_guppi_59103_03721_DIAG_TIC159107668_0016
Figures will appear in in ./raw_gbt_59103_TIC/


#### Run end-to-end test over parameter sweeps

In [2]:

for i_raw in range(n_raw_file):

    raw_file_stem = raw_file_stem_list[i_raw]
    print(f'\n\nRaw file {i_raw+1} of {n_raw_file}: {raw_file_stem}\n\n')

    try:      
        %run ./01_seti_end_to_end.ipynb
    except:
        print(f'\n\nError Occurred for Raw file {raw_file_stem}, skip\n\n')

    if delete_raw_file_after_use & (not local_raw_file_found):
        # delete raw file to limit local file storage
        try:
            print('Deleting ',raw_file_delete_spec)
            os.system('rm '+raw_file_delete_spec)
        except:
            print(' ')

            



Raw file 1 of 8: blc40_guppi_59103_03721_DIAG_TIC159107668_0016


sc2
test_case='sc2'
search_app_name='seticore2', search_z_threshold=20
/home/kmh4005/data_files/raw_files/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.0000.raw
Raw file found in backup directory, no copy, read directly
/mnt/d/data_files/raw_gbt_59103_TIC/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.0000.raw
blc40_guppi_59103_03721_DIAG_TIC159107668_0016
['blc40_guppi_59103_03721_DIAG_TIC159107668_0016']
signal_level = 0.0, n_chirp = 2
f_start1_MHz = nan, f_stop1_MHz = nan,  f_start_incr = nan, df_dt_incr = 20.0
f_start_truth[0] = nan f_start_truth[-1] = nan
drift_rate_truth[0] = -10.0 drift_rate_truth[-1] = 10.0
raw_file_name='/mnt/d/data_files/raw_gbt_59103_TIC/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.0000.raw' found, n_blocks_incr=128, n_blocks=128
raw_file_name='/mnt/d/data_files/raw_gbt_59103_TIC/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.0001.raw' not found, n_raw_files=1, Total blocks=128
raw_file_b

/tmp/ipykernel_2198285/3339333597.py:2: RuntimeWarning: divide by zero encountered in log10
  snr_input_db = 10*np.log10(signal_level**2 /2)  # 1/2 2 pols


rawspec 3.1.1+12@gaf37c70-dirty using librawspec 3.1.1+12@gaf37c70-dirty and cuFFT 11.0.12.1
writing output files in FBH5 format
working stem: /mnt/d/data_files/raw_gbt_59103_TIC/blc40_guppi_59103_03721_DIAG_TIC159107668_0016
opening file: /mnt/d/data_files/raw_gbt_59103_TIC/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.0000.raw
Number of spectra per dump (Nd) = 1
Number of fine channels per coarse channel (nfpc) = 1048576
Chunk dimensions = (1, 1, 1048576)
Cache nslots = 1048577, nbytes = 268435457
opening file: /mnt/d/data_files/raw_gbt_59103_TIC/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.0001.raw [No such file or directory]
output product 0: 16 spectra

Rawspec 87.12 sec 1.36 sec/coarse channel
raw File size =  16385 MB x 1,    256 MB per Coarse Channel
/home/kmh4005/data_files/sg_det_files/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.*.h5
h5_name_list=
['/home/kmh4005/data_files/sg_det_files/blc40_guppi_59103_03721_DIAG_TIC159107668_0016.rawspec.0000.h5']
new_h5_name = '/hom

dedoppler elapsed time: 4s


E(SNR)    nan dB
1 SNR 15.37 dB, 2250.894 MHz, -2.082 Hz/sec
2 SNR 23.09 dB, 2250.596 MHz, -2.732 Hz/sec
3 SNR 24.25 dB, 2250.487 MHz, 1.561 Hz/sec
4 SNR 13.58 dB, 2250.424 MHz, 0.130 Hz/sec
5 SNR 13.19 dB, 2250.414 MHz, -0.260 Hz/sec
6 SNR 13.47 dB, 2250.402 MHz, 0.260 Hz/sec
7 SNR 23.60 dB, 2250.298 MHz, -0.390 Hz/sec
8 SNR 23.44 dB, 2250.280 MHz, 0.520 Hz/sec
9 SNR 28.45 dB, 2250.244 MHz, -1.952 Hz/sec
10 SNR 14.11 dB, 2250.188 MHz, -1.041 Hz/sec
11 SNR 14.05 dB, 2250.168 MHz, 1.561 Hz/sec
12 SNR 13.77 dB, 2250.152 MHz, -1.952 Hz/sec
13 SNR 13.95 dB, 2250.148 MHz, 0.390 Hz/sec
14 SNR 13.88 dB, 2250.135 MHz, 0.781 Hz/sec
15 SNR 15.46 dB, 2250.131 MHz, 2.732 Hz/sec
16 SNR 14.57 dB, 2250.112 MHz, 2.212 Hz/sec
17 SNR 18.50 dB, 2250.090 MHz, -0.130 Hz/sec
18 SNR 15.44 dB, 2250.056 MHz, 0.260 Hz/sec
19 SNR 15.44 dB, 2249.944 MHz, -0.260 Hz/sec
20 SNR 18.51 dB, 2249.910 MHz, 0.130 Hz/sec
21 SNR 14.48 dB, 2249.888 MHz, -2.212 Hz/sec
22 SNR 15.46 dB, 2249.869 MHz, -2.732 Hz/sec
23 SNR 13.88 

...

rm: cannot remove '/home/kmh4005/data_files/raw_files/blc46_guppi_59103_03721_DIAG_TIC159107668_0016.00*.raw': No such file or directory
/tmp/ipykernel_2198285/3339333597.py:2: RuntimeWarning: divide by zero encountered in log10
  snr_input_db = 10*np.log10(signal_level**2 /2)  # 1/2 2 pols


rawspec 3.1.1+12@gaf37c70-dirty using librawspec 3.1.1+12@gaf37c70-dirty and cuFFT 11.0.12.1
writing output files in FBH5 format
working stem: /mnt/d/data_files/raw_gbt_59103_TIC/blc47_guppi_59103_03721_DIAG_TIC159107668_0016
opening file: /mnt/d/data_files/raw_gbt_59103_TIC/blc47_guppi_59103_03721_DIAG_TIC159107668_0016.0000.raw
Number of spectra per dump (Nd) = 1
Number of fine channels per coarse channel (nfpc) = 1048576
Chunk dimensions = (1, 1, 1048576)
Cache nslots = 1048577, nbytes = 268435457
opening file: /mnt/d/data_files/raw_gbt_59103_TIC/blc47_guppi_59103_03721_DIAG_TIC159107668_0016.0001.raw [No such file or directory]
output product 0: 16 spectra

Rawspec 94.92 sec 1.48 sec/coarse channel
raw File size =  16385 MB x 1,    256 MB per Coarse Channel
/home/kmh4005/data_files/sg_det_files/blc47_guppi_59103_03721_DIAG_TIC159107668_0016.*.h5
h5_name_list=
['/home/kmh4005/data_files/sg_det_files/blc47_guppi_59103_03721_DIAG_TIC159107668_0016.rawspec.0000.h5']
new_h5_name = '/hom

dedoppler elapsed time: 4s


E(SNR)    nan dB
1 SNR 22.57 dB, 937.552 MHz, 5.464 Hz/sec
2 SNR 14.48 dB, 937.547 MHz, -9.758 Hz/sec
3 SNR 17.55 dB, 937.536 MHz, 1.301 Hz/sec
4 SNR 14.07 dB, 937.533 MHz, 7.676 Hz/sec
5 SNR 21.83 dB, 937.529 MHz, 8.457 Hz/sec
6 SNR 19.79 dB, 937.518 MHz, 2.212 Hz/sec
7 SNR 13.57 dB, 937.517 MHz, -7.286 Hz/sec
8 SNR 13.30 dB, 937.512 MHz, 1.952 Hz/sec
9 SNR 16.31 dB, 937.506 MHz, -3.253 Hz/sec
10 SNR 13.46 dB, 937.483 MHz, -1.691 Hz/sec
11 SNR 16.34 dB, 937.462 MHz, 2.732 Hz/sec
12 SNR 19.57 dB, 937.439 MHz, -7.936 Hz/sec
13 SNR 13.09 dB, 937.433 MHz, 2.212 Hz/sec
14 SNR 13.16 dB, 937.409 MHz, -7.546 Hz/sec
15 SNR 13.58 dB, 937.329 MHz, 1.952 Hz/sec
16 SNR 22.62 dB, 931.531 MHz, -0.911 Hz/sec
17 SNR 21.26 dB, 929.985 MHz, 0.390 Hz/sec
18 SNR 19.92 dB, 916.679 MHz, -0.260 Hz/sec
19 SNR 14.88 dB, 916.633 MHz, 1.821 Hz/sec
20 SNR 17.65 dB, 903.771 MHz, -1.041 Hz/sec
21 SNR 20.56 dB, 900.841 MHz, -1.041 Hz/sec
22 SNR 15.56 dB, 875.057 MHz, 3.643 Hz/sec
23 SNR 13.15 dB, 875.035 MHz, -1.431

rm: cannot remove '/home/kmh4005/data_files/raw_files/blc47_guppi_59103_03721_DIAG_TIC159107668_0016.00*.raw': No such file or directory


In [3]:
import os

print('All Runs Done')

# Beep 2 times in WSL
os.system("powershell.exe '[console]::beep(261.6,700)'")
os.system("powershell.exe '[console]::beep(261.6,700)'")


All Runs Done


0